In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import sklearn.preprocessing as sk
import seaborn as sns
from sklearn import metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric
from torch.utils.data.sampler import WeightedRandomSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import random
from random import randint
from sklearn.model_selection import StratifiedKFold

save_results_to = '/home/hnoghabi/EGFR/'
torch.manual_seed(42)
random.seed(42)

GDSCE = pd.read_csv("GDSC_exprs.z.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")
GDSCE = pd.DataFrame.transpose(GDSCE)

GDSCM = pd.read_csv("GDSC_mutations.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ".")
GDSCM = pd.DataFrame.transpose(GDSCM)
GDSCM = GDSCM.loc[:,~GDSCM.columns.duplicated()]

GDSCC = pd.read_csv("GDSC_CNA.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ".")
GDSCC.drop_duplicates(keep='last')
GDSCC = pd.DataFrame.transpose(GDSCC)
GDSCC = GDSCC.loc[:,~GDSCC.columns.duplicated()]

PDXEerlo = pd.read_csv("PDX_exprs.Erlotinib.eb_with.GDSC_exprs.Erlotinib.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXEerlo = pd.DataFrame.transpose(PDXEerlo)
PDXMerlo = pd.read_csv("PDX_mutations.Erlotinib.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXMerlo = pd.DataFrame.transpose(PDXMerlo)
PDXCerlo = pd.read_csv("PDX_CNV.Erlotinib.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXCerlo.drop_duplicates(keep='last')
PDXCerlo = pd.DataFrame.transpose(PDXCerlo)
PDXCerlo = PDXCerlo.loc[:,~PDXCerlo.columns.duplicated()]

PDXEcet = pd.read_csv("PDX_exprs.Cetuximab.eb_with.GDSC_exprs.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXEcet = pd.DataFrame.transpose(PDXEcet)
PDXMcet = pd.read_csv("PDX_mutations.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXMcet = pd.DataFrame.transpose(PDXMcet)
PDXCcet = pd.read_csv("PDX_CNV.Cetuximab.tsv", 
                   sep = "\t", index_col=0, decimal = ",")
PDXCcet.drop_duplicates(keep='last')
PDXCcet = pd.DataFrame.transpose(PDXCcet)
PDXCcet = PDXCcet.loc[:,~PDXCcet.columns.duplicated()]

selector = VarianceThreshold(0.05)
selector.fit_transform(GDSCE)
GDSCE = GDSCE[GDSCE.columns[selector.get_support(indices=True)]]

GDSCM = GDSCM.fillna(0)
GDSCM[GDSCM != 0.0] = 1
GDSCC = GDSCC.fillna(0)
GDSCC[GDSCC != 0.0] = 1

ls = GDSCE.columns.intersection(GDSCM.columns)
ls = ls.intersection(GDSCC.columns)
ls = ls.intersection(PDXEerlo.columns)
ls = ls.intersection(PDXMerlo.columns)
ls = ls.intersection(PDXCerlo.columns)
ls = ls.intersection(PDXEcet.columns)
ls = ls.intersection(PDXMcet.columns)
ls = ls.intersection(PDXCcet.columns)
ls2 = GDSCE.index.intersection(GDSCM.index)
ls2 = ls2.intersection(GDSCC.index)
ls3 = PDXEerlo.index.intersection(PDXMerlo.index)
ls3 = ls3.intersection(PDXCerlo.index)
ls4 = PDXEcet.index.intersection(PDXMcet.index)
ls4 = ls4.intersection(PDXCcet.index)
ls = pd.unique(ls)

PDXEerlo = PDXEerlo.loc[ls3,ls]
PDXMerlo = PDXMerlo.loc[ls3,ls]
PDXCerlo = PDXCerlo.loc[ls3,ls]
PDXEcet = PDXEcet.loc[ls4,ls]
PDXMcet = PDXMcet.loc[ls4,ls]
PDXCcet = PDXCcet.loc[ls4,ls]
GDSCE = GDSCE.loc[:,ls]
GDSCM = GDSCM.loc[:,ls]
GDSCC = GDSCC.loc[:,ls]

GDSCR = pd.read_csv("GDSC_response.EGFRi.tsv", 
                    sep = "\t", index_col=0, decimal = ",")

GDSCR.rename(mapper = str, axis = 'index', inplace = True)

d = {"R":0,"S":1}
GDSCR["response"] = GDSCR.loc[:,"response"].apply(lambda x: d[x])

responses = GDSCR
drugs = set(responses["drug"].values)
exprs_z = GDSCE
cna = GDSCC
mut = GDSCM
expression_zscores = []
CNA=[]
mutations = []
for drug in drugs:
    samples = responses.loc[responses["drug"]==drug,:].index.values
    e_z = exprs_z.loc[samples,:]
    c = cna.loc[samples,:]
    m = mut.loc[samples,:]
    m = mut.loc[samples,:]
    # next 3 rows if you want non-unique sample names
    e_z.rename(lambda x : str(x)+"_"+drug, axis = "index", inplace=True)
    c.rename(lambda x : str(x)+"_"+drug, axis = "index", inplace=True)
    m.rename(lambda x : str(x)+"_"+drug, axis = "index", inplace=True)
    expression_zscores.append(e_z)
    CNA.append(c)
    mutations.append(m)
responses.index = responses.index.values +"_"+responses["drug"].values
GDSCEv2 = pd.concat(expression_zscores, axis =0 )
GDSCCv2 = pd.concat(CNA, axis =0 )
GDSCMv2 = pd.concat(mutations, axis =0 )
GDSCRv2 = responses

ls2 = GDSCEv2.index.intersection(GDSCMv2.index)
ls2 = ls2.intersection(GDSCCv2.index)
GDSCEv2 = GDSCEv2.loc[ls2,:]
GDSCMv2 = GDSCMv2.loc[ls2,:]
GDSCCv2 = GDSCCv2.loc[ls2,:]
GDSCRv2 = GDSCRv2.loc[ls2,:]

Y = GDSCRv2['response'].values

PDXRcet = pd.read_csv("PDX_response.Cetuximab.tsv", 
                       sep = "\t", index_col=0, decimal = ",")
PDXRcet.loc[PDXRcet.iloc[:,0] == 'R'] = 0
PDXRcet.loc[PDXRcet.iloc[:,0] == 'S'] = 1
PDXRcet = PDXRcet.loc[ls4,:]
Ytscet = PDXRcet['response'].values    

PDXRerlo = pd.read_csv("PDX_response.Erlotinib.tsv", 
                       sep = "\t", index_col=0, decimal = ",")
PDXRerlo.loc[PDXRerlo.iloc[:,0] == 'R'] = 0
PDXRerlo.loc[PDXRerlo.iloc[:,0] == 'S'] = 1
PDXRerlo = PDXRerlo.loc[ls3,:]
Ytserlo = PDXRerlo['response'].values  

hdm1 = 32
hdm2 = 16
hdm3 = 256
rate1 = 0.5
rate2 = 0.8
rate3 = 0.5
rate4 = 0.3

scalerGDSC = sk.StandardScaler()
scalerGDSC.fit(GDSCEv2.values)
X_trainE = scalerGDSC.transform(GDSCEv2.values)
X_testEerlo = scalerGDSC.transform(PDXEerlo.values)    
X_testEcet = scalerGDSC.transform(PDXEcet.values)    

X_trainM = np.nan_to_num(GDSCMv2.values)
X_trainC = np.nan_to_num(GDSCCv2.values)
X_testMerlo = np.nan_to_num(PDXMerlo.values)
X_testCerlo = np.nan_to_num(PDXCerlo.values)
X_testMcet = np.nan_to_num(PDXMcet.values)
X_testCcet = np.nan_to_num(PDXCcet.values)

TX_testEerlo = torch.FloatTensor(X_testEerlo)
TX_testMerlo = torch.FloatTensor(X_testMerlo)
TX_testCerlo = torch.FloatTensor(X_testCerlo)
ty_testEerlo = torch.FloatTensor(Ytserlo.astype(int))

TX_testEcet = torch.FloatTensor(X_testEcet)
TX_testMcet = torch.FloatTensor(X_testMcet)
TX_testCcet = torch.FloatTensor(X_testCcet)
ty_testEcet = torch.FloatTensor(Ytscet.astype(int))

n_sampE, IE_dim = X_trainE.shape
n_sampM, IM_dim = X_trainM.shape
n_sampC, IC_dim = X_trainC.shape

h_dim1 = hdm1
h_dim2 = hdm2
h_dim3 = hdm3        
Z_in = h_dim1 + h_dim2 + h_dim3

costtr = []
auctr = []
costts = []
aucts = []

class AEE(nn.Module):
    def __init__(self):
        super(AEE, self).__init__()
        self.EnE = torch.nn.Sequential(
            nn.Linear(IE_dim, h_dim1),
            nn.BatchNorm1d(h_dim1),
            nn.ReLU(),
            nn.Dropout(rate1))
    def forward(self, x):
        output = self.EnE(x)
        return output

class AEM(nn.Module):
    def __init__(self):
        super(AEM, self).__init__()
        self.EnM = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim2),
            nn.BatchNorm1d(h_dim2),
            nn.ReLU(),
            nn.Dropout(rate2))
    def forward(self, x):
        output = self.EnM(x)
        return output    


class AEC(nn.Module):
    def __init__(self):
        super(AEC, self).__init__()
        self.EnC = torch.nn.Sequential(
            nn.Linear(IM_dim, h_dim3),
            nn.BatchNorm1d(h_dim3),
            nn.ReLU(),
            nn.Dropout(rate3))
    def forward(self, x):
        output = self.EnC(x)
        return output       

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.FC = torch.nn.Sequential(
            nn.Linear(Z_in, 1),
            nn.Dropout(rate4),
            nn.Sigmoid())
    def forward(self, x):
        return self.FC(x)

torch.cuda.manual_seed_all(42)

AutoencoderE = torch.load('EGFRv2Exprs.pt')
AutoencoderM = torch.load('EGFRv2Mut.pt')
AutoencoderC = torch.load('EGFRv2CNA.pt')

Clas = torch.load('EGFRv2Class.pt')

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

ZEX = AutoencoderE(torch.FloatTensor(X_trainE))
ZMX = AutoencoderM(torch.FloatTensor(X_trainM))
ZCX = AutoencoderC(torch.FloatTensor(X_trainC))
ZTX = torch.cat((ZEX, ZMX, ZCX), 1)
ZTX = F.normalize(ZTX, p=2, dim=0)
PredX = Clas(ZTX)
AUCt = roc_auc_score(Y, PredX.detach().numpy())
print(AUCt)

ZETerlo = AutoencoderE(TX_testEerlo)
ZMTerlo = AutoencoderM(TX_testMerlo)
ZCTerlo = AutoencoderC(TX_testCerlo)
ZTTerlo = torch.cat((ZETerlo, ZMTerlo, ZCTerlo), 1)
ZTTerlo = F.normalize(ZTTerlo, p=2, dim=0)
PredTerlo = Clas(ZTTerlo)
AUCterlo = roc_auc_score(Ytserlo, PredTerlo.detach().numpy())
print(AUCterlo)

ZETcet = AutoencoderE(TX_testEcet)
ZMTcet = AutoencoderM(TX_testMcet)
ZCTcet = AutoencoderC(TX_testCcet)
ZTTcet = torch.cat((ZETcet, ZMTcet, ZCTcet), 1)
ZTTcet = F.normalize(ZTTcet, p=2, dim=0)
PredTcet = Clas(ZTTcet)
AUCtcet = roc_auc_score(Ytscet, PredTcet.detach().numpy())
print(AUCtcet)

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0.9440567436313729
0.7222222222222222
0.8


In [2]:
PRADE = pd.read_csv("TCGA-PRAD_exprs.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PRADE = pd.DataFrame.transpose(PRADE)

PRADM = pd.read_csv("TCGA-PRAD_mutations.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PRADM = pd.DataFrame.transpose(PRADM)
PRADM = PRADM.loc[:,~PRADM.columns.duplicated()]

PRADC = pd.read_csv("TCGA-PRAD_CNA.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PRADC = pd.DataFrame.transpose(PRADC)
PRADC = PRADC.loc[:,~PRADC.columns.duplicated()]

PRADM = PRADM.fillna(0)
PRADM[PRADM != 0.0] = 1
PRADC = PRADC.fillna(0)
PRADC[PRADC != 0.0] = 1

#PRADE.rename(lambda x : x[0:11], axis = "index", inplace=True)  
#PRADM.rename(lambda x : x[0:11], axis = "index", inplace=True)   
#PRADC.rename(lambda x : x[0:11], axis = "index", inplace=True)   

lsPRAD = PRADE.index.intersection(PRADM.index)
lsPRAD = lsPRAD.intersection(PRADC.index)
lsPRAD = pd.unique(lsPRAD)

PRADE = PRADE.loc[lsPRAD,ls]
PRADM = PRADM.loc[lsPRAD,ls]
PRADC = PRADC.loc[lsPRAD,ls]

print(PRADE.shape)
print(PRADM.shape)
print(PRADC.shape)

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

PRADE2 = np.nan_to_num(PRADE.values)
PRADM2 = np.nan_to_num(PRADM.values)
PRADC2 = np.nan_to_num(PRADC.values)

NPRADE2 = scalerGDSC.transform(PRADE2)    

PRADexprs = torch.FloatTensor(NPRADE2)
PRADmut = torch.FloatTensor(PRADM2)
PRADcna = torch.FloatTensor(PRADC2)

PRADZE = AutoencoderE(PRADexprs)
PRADZM = AutoencoderM(PRADmut)
PRADZC = AutoencoderC(PRADcna)

PRADZT = torch.cat((PRADZE, PRADZM, PRADZC), 1)
PRADZTX = F.normalize(PRADZT, p=2, dim=0)
PredPRAD = Clas(PRADZTX)

#print(PredPRAD.detach().numpy())

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/inde

(492, 13081)
(492, 13081)
(492, 13081)


In [3]:
KIRPE = pd.read_csv("TCGA-KIRP_exprs.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
KIRPE = pd.DataFrame.transpose(KIRPE)

KIRPM = pd.read_csv("TCGA-KIRP_mutations.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
KIRPM = pd.DataFrame.transpose(KIRPM)
KIRPM = KIRPM.loc[:,~KIRPM.columns.duplicated()]

KIRPC = pd.read_csv("TCGA-KIRP_CNA.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
KIRPC = pd.DataFrame.transpose(KIRPC)
KIRPC = KIRPC.loc[:,~KIRPC.columns.duplicated()]

KIRPM = KIRPM.fillna(0)
KIRPM[KIRPM != 0.0] = 1
KIRPC = KIRPC.fillna(0)
KIRPC[KIRPC != 0.0] = 1

#KIRPE.rename(lambda x : x[0:11], axis = "index", inplace=True)  
#KIRPM.rename(lambda x : x[0:11], axis = "index", inplace=True)   
#KIRPC.rename(lambda x : x[0:11], axis = "index", inplace=True)   

lsKIRP = KIRPE.index.intersection(KIRPM.index)
lsKIRP = lsKIRP.intersection(KIRPC.index)
lsKIRP = pd.unique(lsKIRP)

KIRPE = KIRPE.loc[lsKIRP,ls]
KIRPM = KIRPM.loc[lsKIRP,ls]
KIRPC = KIRPC.loc[lsKIRP,ls]

print(KIRPE.shape)
print(KIRPM.shape)
print(KIRPC.shape)

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

KIRPE2 = np.nan_to_num(KIRPE.values)
KIRPM2 = np.nan_to_num(KIRPM.values)
KIRPC2 = np.nan_to_num(KIRPC.values)

NKIRPE2 = scalerGDSC.transform(KIRPE2)    

KIRPexprs = torch.FloatTensor(NKIRPE2)
KIRPmut = torch.FloatTensor(KIRPM2)
KIRPcna = torch.FloatTensor(KIRPC2)

KIRPZE = AutoencoderE(KIRPexprs)
KIRPZM = AutoencoderM(KIRPmut)
KIRPZC = AutoencoderC(KIRPcna)

KIRPZT = torch.cat((KIRPZE, KIRPZM, KIRPZC), 1)
KIRPZTX = F.normalize(KIRPZT, p=2, dim=0)
PredKIRP = Clas(KIRPZTX)

#print(PredKIRP.detach().numpy())

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/inde

(161, 13081)
(161, 13081)
(161, 13081)


In [4]:
BLCAE = pd.read_csv("TCGA-BLCA_exprs.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
BLCAE = pd.DataFrame.transpose(BLCAE)

BLCAM = pd.read_csv("TCGA-BLCA_mutations.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
BLCAM = pd.DataFrame.transpose(BLCAM)
BLCAM = BLCAM.loc[:,~BLCAM.columns.duplicated()]

BLCAC = pd.read_csv("TCGA-BLCA_CNA.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
BLCAC = pd.DataFrame.transpose(BLCAC)
BLCAC = BLCAC.loc[:,~BLCAC.columns.duplicated()]

BLCAM = BLCAM.fillna(0)
BLCAM[BLCAM != 0.0] = 1
BLCAC = BLCAC.fillna(0)
BLCAC[BLCAC != 0.0] = 1

#BLCAE.rename(lambda x : x[0:11], axis = "index", inplace=True)  
#BLCAM.rename(lambda x : x[0:11], axis = "index", inplace=True)   
#BLCAC.rename(lambda x : x[0:11], axis = "index", inplace=True)   

lsBLCA = BLCAE.index.intersection(BLCAM.index)
lsBLCA = lsBLCA.intersection(BLCAC.index)
lsBLCA = pd.unique(lsBLCA)

BLCAE = BLCAE.loc[lsBLCA,ls]
BLCAM = BLCAM.loc[lsBLCA,ls]
BLCAC = BLCAC.loc[lsBLCA,ls]

print(BLCAE.shape)
print(BLCAM.shape)
print(BLCAC.shape)

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

BLCAE2 = np.nan_to_num(BLCAE.values)
BLCAM2 = np.nan_to_num(BLCAM.values)
BLCAC2 = np.nan_to_num(BLCAC.values)

NBLCAE2 = scalerGDSC.transform(BLCAE2)    

BLCAexprs = torch.FloatTensor(NBLCAE2)
BLCAmut = torch.FloatTensor(BLCAM2)
BLCAcna = torch.FloatTensor(BLCAC2)

BLCAZE = AutoencoderE(BLCAexprs)
BLCAZM = AutoencoderM(BLCAmut)
BLCAZC = AutoencoderC(BLCAcna)

BLCAZT = torch.cat((BLCAZE, BLCAZM, BLCAZC), 1)
BLCAZTX = F.normalize(BLCAZT, p=2, dim=0)
PredBLCA = Clas(BLCAZTX)

#print(PredBLCA.detach().numpy())

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/inde

(123, 13081)
(123, 13081)
(123, 13081)


In [5]:
BRCAE = pd.read_csv("TCGA-BRCA_exprs.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
BRCAE = pd.DataFrame.transpose(BRCAE)

BRCAM = pd.read_csv("TCGA-BRCA_mutations.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
BRCAM = pd.DataFrame.transpose(BRCAM)
BRCAM = BRCAM.loc[:,~BRCAM.columns.duplicated()]

BRCAC = pd.read_csv("TCGA-BRCA_CNA.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
BRCAC = pd.DataFrame.transpose(BRCAC)
BRCAC = BRCAC.loc[:,~BRCAC.columns.duplicated()]

BRCAM = BRCAM.fillna(0)
BRCAM[BRCAM != 0.0] = 1
BRCAC = BRCAC.fillna(0)
BRCAC[BRCAC != 0.0] = 1

#BRCAE.rename(lambda x : x[0:11], axis = "index", inplace=True)  
#BRCAM.rename(lambda x : x[0:11], axis = "index", inplace=True)   
#BRCAC.rename(lambda x : x[0:11], axis = "index", inplace=True)   

lsBRCA = BRCAE.index.intersection(BRCAM.index)
lsBRCA = lsBRCA.intersection(BRCAC.index)
lsBRCA = pd.unique(lsBRCA)

BRCAE = BRCAE.loc[lsBRCA,ls]
BRCAM = BRCAM.loc[lsBRCA,ls]
BRCAC = BRCAC.loc[lsBRCA,ls]

print(BRCAE.shape)
print(BRCAM.shape)
print(BRCAC.shape)

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

BRCAE2 = np.nan_to_num(BRCAE.values)
BRCAM2 = np.nan_to_num(BRCAM.values)
BRCAC2 = np.nan_to_num(BRCAC.values)

NBRCAE2 = scalerGDSC.transform(BRCAE2)    

BRCAexprs = torch.FloatTensor(NBRCAE2)
BRCAmut = torch.FloatTensor(BRCAM2)
BRCAcna = torch.FloatTensor(BRCAC2)

BRCAZE = AutoencoderE(BRCAexprs)
BRCAZM = AutoencoderM(BRCAmut)
BRCAZC = AutoencoderC(BRCAcna)

BRCAZT = torch.cat((BRCAZE, BRCAZM, BRCAZC), 1)
BRCAZTX = F.normalize(BRCAZT, p=2, dim=0)
PredBRCA = Clas(BRCAZTX)

#print(PredBRCA.detach().numpy())

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/inde

(921, 13081)
(921, 13081)
(921, 13081)


In [6]:
PAADE = pd.read_csv("TCGA-PAAD_exprs.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PAADE = pd.DataFrame.transpose(PAADE)

PAADM = pd.read_csv("TCGA-PAAD_mutations.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PAADM = pd.DataFrame.transpose(PAADM)
PAADM = PAADM.loc[:,~PAADM.columns.duplicated()]

PAADC = pd.read_csv("TCGA-PAAD_CNA.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
PAADC = pd.DataFrame.transpose(PAADC)
PAADC = PAADC.loc[:,~PAADC.columns.duplicated()]

PAADM = PAADM.fillna(0)
PAADM[PAADM != 0.0] = 1
PAADC = PAADC.fillna(0)
PAADC[PAADC != 0.0] = 1

#PAADE.rename(lambda x : x[0:11], axis = "index", inplace=True)  
#PAADM.rename(lambda x : x[0:11], axis = "index", inplace=True)   
#PAADC.rename(lambda x : x[0:11], axis = "index", inplace=True)   

lsPAAD = PAADE.index.intersection(PAADM.index)
lsPAAD = lsPAAD.intersection(PAADC.index)
lsPAAD = pd.unique(lsPAAD)

PAADE = PAADE.loc[lsPAAD,ls]
PAADM = PAADM.loc[lsPAAD,ls]
PAADC = PAADC.loc[lsPAAD,ls]

print(PAADE.shape)
print(PAADM.shape)
print(PAADC.shape)

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

PAADE2 = np.nan_to_num(PAADE.values)
PAADM2 = np.nan_to_num(PAADM.values)
PAADC2 = np.nan_to_num(PAADC.values)

NPAADE2 = scalerGDSC.transform(PAADE2)    

PAADexprs = torch.FloatTensor(NPAADE2)
PAADmut = torch.FloatTensor(PAADM2)
PAADcna = torch.FloatTensor(PAADC2)

PAADZE = AutoencoderE(PAADexprs)
PAADZM = AutoencoderM(PAADmut)
PAADZC = AutoencoderC(PAADcna)

PAADZT = torch.cat((PAADZE, PAADZM, PAADZC), 1)
PAADZTX = F.normalize(PAADZT, p=2, dim=0)
PredPAAD = Clas(PAADZTX)

#print(PredPAAD.detach().numpy())

(130, 13081)
(130, 13081)
(130, 13081)


/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/inde

In [7]:
LUADE = pd.read_csv("TCGA-LUAD_exprs.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
LUADE = pd.DataFrame.transpose(LUADE)

LUADM = pd.read_csv("TCGA-LUAD_mutations.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
LUADM = pd.DataFrame.transpose(LUADM)
LUADM = LUADM.loc[:,~LUADM.columns.duplicated()]

LUADC = pd.read_csv("TCGA-LUAD_CNA.tsv", 
                   sep = "\t", index_col=0, decimal = ".")
LUADC = pd.DataFrame.transpose(LUADC)
LUADC = LUADC.loc[:,~LUADC.columns.duplicated()]

LUADM = LUADM.fillna(0)
LUADM[LUADM != 0.0] = 1
LUADC = LUADC.fillna(0)
LUADC[LUADC != 0.0] = 1

#LUADE.rename(lambda x : x[0:11], axis = "index", inplace=True)  
#LUADM.rename(lambda x : x[0:11], axis = "index", inplace=True)   
#LUADC.rename(lambda x : x[0:11], axis = "index", inplace=True)   

lsLUAD = LUADE.index.intersection(LUADM.index)
lsLUAD = lsLUAD.intersection(LUADC.index)
lsLUAD = pd.unique(lsLUAD)

LUADE = LUADE.loc[lsLUAD,ls]
LUADM = LUADM.loc[lsLUAD,ls]
LUADC = LUADC.loc[lsLUAD,ls]

print(LUADE.shape)
print(LUADM.shape)
print(LUADC.shape)

AutoencoderE.eval()
AutoencoderM.eval()
AutoencoderC.eval()
Clas.eval()

LUADE2 = np.nan_to_num(LUADE.values)
LUADM2 = np.nan_to_num(LUADM.values)
LUADC2 = np.nan_to_num(LUADC.values)

NLUADE2 = scalerGDSC.transform(LUADE2)    

LUADexprs = torch.FloatTensor(NLUADE2)
LUADmut = torch.FloatTensor(LUADM2)
LUADcna = torch.FloatTensor(LUADC2)

LUADZE = AutoencoderE(LUADexprs)
LUADZM = AutoencoderM(LUADmut)
LUADZC = AutoencoderC(LUADcna)

LUADZT = torch.cat((LUADZE, LUADZM, LUADZC), 1)
LUADZTX = F.normalize(LUADZT, p=2, dim=0)
PredLUAD = Clas(LUADZTX)

#print(PredLUAD.detach().numpy())

/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/hnoghabi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/inde

(475, 13081)
(475, 13081)
(475, 13081)


In [8]:
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr
import statsmodels.api as sm
from mne.stats import bonferroni_correction

lsEGFR = [10000, 102, 10252, 10253,10254,1026,1027,107,108,109,111,11140,112,113,114,1147,115,117145,1173,1175,1211,1213,1385,1445,156,160,161,163,1950,1956,196883,2060,207,208,2308,2309,23239,2475,253260,2549,26018,2885,2931,29924,30011,3164,3265,3320,3709,3710,3845,4193,4303,4893,5136,5153,5170,5290,5295,5335,5566,5567,5568,5573,5575,5576,5577,5578,5580,5581,5582,55824,5594,5595,5604,5605,572,5728,57761,58513,5894,6199,6233,64223,6456,6464,6654,6714,6868,7249,728590,729120,730418,7311,731292,7529,79109,801,8027,8038,805,808,814,842,84335,867,9146,983,998]

#lsEGFR = [10000,1026,1027,10298,10718,1398,1399,145957,1839,1950,1956,1978,2002,2064,2065,2066,2069,207,208,23533,23642,2475,25,2549,25759,27,2885,2932,3084,3265,369,3725,374,3845,399694,4609,4690,4893,5058,5062,5063,5290,5291,5293,5294,5295,5296,5335,53358,5336,5578,5579,5582,5594,5595,5599,5601,5602,5604,5605,5609,56924,57144,572,5747,5894,6198,6199,6416,6464,6654,6655,6714,673,6776,6777,685,7039,815,816,817,818,8440,8503,867,868,9542]

In [12]:
listEGFR = PRADE.columns.intersection(lsEGFR)
PRADEEGFR = PRADE[listEGFR]
PRADMEGFR = PRADM[listEGFR]
PRADCEGFR = PRADC[listEGFR]

In [13]:
X = PRADEEGFR
y = PredPRAD.detach().numpy()

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     3331.
Date:                Sat, 12 Jan 2019   Prob (F-statistic):               0.00
Time:                        18:16:35   Log-Likelihood:                 1340.8
No. Observations:                 492   AIC:                            -2480.
Df Residuals:                     391   BIC:                            -2056.
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
102            0.0038      0.003      1.171      0.242      -0.003       0.010
107           -0.0015      0.001     -0.982      0.327      -0.004       0.001
108           -0.0006      0.002     -0.377      0.706      -0.004       0.003
109           -0.0021      0.004     -0.585      0.559      -0.009       0.005
111           -0.0038      0.003     -1.432      0.153      -0.009       0.001
112           -0.0092      0.004     -2.553      0.011      -0.016      -0.002
113            0.0046      0.003      1.325      0.186      -0.002       0.011
114            0.0184      0.021      0.868      0.386      -0.023       0.060
115           -0.0002      0.004     -0.047      0.963      -0.007       0.007
160            0.0020      0.004      0.564      0.573      -0.005       0.009
161            0.0081      0.006      1.424      0.155      -0.003       0.019
163            0.0006      0.004      0.142      0.887      -0.008       0.009
207            0.0157      0.004      3.698      0.000       0.007       0.024
208            0.0044      0.006      0.792      0.429      -0.007       0.015
572            0.0089      0.004      1.975      0.049    3.82e-05       0.018
801           -0.0057      0.003     -1.772      0.077      -0.012       0.001
805            0.0053      0.004      1.280      0.201      -0.003       0.013
808            0.0055      0.005      1.141      0.255      -0.004       0.015
814           -0.0012      0.003     -0.477      0.633      -0.006       0.004
842           -0.0072      0.004     -1.804      0.072      -0.015       0.001
867            0.0034      0.005      0.666      0.506      -0.007       0.014
983            0.0062      0.002      3.919      0.000       0.003       0.009
998           -0.0097      0.005     -1.854      0.064      -0.020       0.001
1026           0.0025      0.001      1.717      0.087      -0.000       0.005
1027          -0.0033      0.003     -1.224      0.222      -0.009       0.002
1147          -0.0025      0.004     -0.635      0.526      -0.010       0.005
1173           0.0166      0.005      3.472      0.001       0.007       0.026
1175          -0.0029      0.003     -0.927      0.355      -0.009       0.003
1211           0.0137      0.004      3.271      0.001       0.005       0.022
1213          -0.0176      0.005     -3.238      0.001      -0.028      -0.007
1385          -0.0272      0.006     -4.807      0.000      -0.038      -0.016
1445          -0.0190      0.005     -3.692      0.000      -0.029      -0.009
1950           0.0015      0.001      1.262      0.208      -0.001       0.004
1956           0.0091      0.003      3.223      0.001       0.004       0.015
2060          -0.0084      0.005     -1.800      0.073      -0.018       0.001
2308          -0.0026      0.004     -0.730      0.466      -0.010       0.004
23

In [14]:
print(bonferroni_correction(model.pvalues, alpha=0.05))

(array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True]), array([2.44677740e+01, 3.29985438e+01, 7.13092918e+01, 5.64211888e+01,
       1.54509011e+01, 1.11658624e+00, 1.87812262e+01, 3.89558813e+01,
       9.72150779e+01, 5.78711869e+01, 1.56777500e+01, 8.9568282

In [15]:
listEGFR = KIRPE.columns.intersection(lsEGFR)
KIRPEEGFR = KIRPE[listEGFR]
KIRPMEGFR = KIRPM[listEGFR]
KIRPCEGFR = KIRPC[listEGFR]   

In [16]:
X = KIRPEEGFR
y = PredKIRP.detach().numpy()

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     356.2
Date:                Sat, 12 Jan 2019   Prob (F-statistic):           2.84e-62
Time:                        18:16:45   Log-Likelihood:                 409.04
No. Observations:                 161   AIC:                            -616.1
Df Residuals:                      60   BIC:                            -304.9
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
102           -0.0203      0.011     -1.884      0.064      -0.042       0.001
107            0.0050      0.015      0.340      0.735      -0.024       0.034
108           -0.0149      0.006     -2.486      0.016      -0.027      -0.003
109            0.0090      0.008      1.139      0.259      -0.007       0.025
111            0.0058      0.005      1.084      0.283      -0.005       0.016
112            0.0015      0.013      0.118      0.906      -0.024       0.027
113           -0.0260      0.012     -2.093      0.041      -0.051      -0.001
114           -0.0026      0.014     -0.180      0.858      -0.031       0.026
115            0.0049      0.012      0.412      0.682      -0.019       0.029
160            0.0032      0.018      0.179      0.859      -0.032       0.039
161           -0.0461      0.019     -2.433      0.018      -0.084      -0.008
163            0.0056      0.019      0.295      0.769      -0.032       0.044
207           -0.0264      0.016     -1.696      0.095      -0.058       0.005
208            0.0413      0.016      2.571      0.013       0.009       0.073
572            0.0455      0.019      2.357      0.022       0.007       0.084
801           -0.0083      0.013     -0.614      0.542      -0.035       0.019
805           -0.0226      0.023     -0.972      0.335      -0.069       0.024
808           -0.0373      0.018     -2.121      0.038      -0.072      -0.002
814            0.0115      0.020      0.561      0.577      -0.029       0.052
842           -0.0297      0.015     -2.021      0.048      -0.059      -0.000
867           -0.0059      0.024     -0.248      0.805      -0.053       0.042
983            0.0012      0.007      0.169      0.867      -0.013       0.016
998            0.0193      0.024      0.822      0.414      -0.028       0.066
1026          -0.0095      0.008     -1.213      0.230      -0.025       0.006
1027          -0.0049      0.012     -0.417      0.678      -0.028       0.019
1147           0.0307      0.020      1.518      0.134      -0.010       0.071
1173           0.0353      0.019      1.827      0.073      -0.003       0.074
1175           0.0282      0.024      1.194      0.237      -0.019       0.075
1211           0.0049      0.015      0.337      0.738      -0.024       0.034
1213          -0.0081      0.017     -0.484      0.630      -0.041       0.025
1385          -0.0046      0.020     -0.234      0.816      -0.044       0.035
1445           0.0122      0.019      0.658      0.513      -0.025       0.049
1950           0.0033      0.008      0.419      0.677      -0.012       0.019
1956           0.0367      0.010      3.749      0.000       0.017       0.056
2060          -0.0484      0.024     -2.035      0.046      -0.096      -0.001
2308          -0.0066      0.013     -0.519      0.606      -0.032       0.019
23

In [17]:
print(bonferroni_correction(model.pvalues, alpha=0.05))

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([6.50924133e+00, 7.42508317e+01, 1.58961376e+00, 2.61829337e+01,
       2.85630855e+01, 9.15220271e+01, 4.09864261e+00, 8.66437911e+01,
       6.88992687e+01, 8.67118407e+01, 1.81588898e+00, 7.7648348

In [18]:
listEGFR = BLCAE.columns.intersection(lsEGFR)
BLCAEEGFR = BLCAE[listEGFR]
BLCAMEGFR = BLCAM[listEGFR]
BLCACEGFR = BLCAC[listEGFR]   

In [19]:
X = BLCAEEGFR
y = PredBLCA.detach().numpy()

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     91.05
Date:                Sat, 12 Jan 2019   Prob (F-statistic):           4.86e-18
Time:                        18:16:52   Log-Likelihood:                 291.96
No. Observations:                 123   AIC:                            -381.9
Df Residuals:                      22   BIC:                            -97.88
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
102            0.0508      0.025      2.072      0.050   -4.41e-05       0.102
107            0.0081      0.011      0.733      0.471      -0.015       0.031
108            0.0587      0.039      1.503      0.147      -0.022       0.140
109            0.0054      0.021      0.253      0.803      -0.039       0.050
111            0.0114      0.018      0.620      0.541      -0.027       0.050
112           -0.0243      0.023     -1.037      0.311      -0.073       0.024
113            0.0279      0.022      1.292      0.210      -0.017       0.073
114            0.4896      0.285      1.715      0.100      -0.102       1.081
115           -0.0062      0.029     -0.215      0.831      -0.066       0.054
160           -0.0502      0.034     -1.455      0.160      -0.122       0.021
161            0.0540      0.036      1.521      0.143      -0.020       0.128
163           -0.0213      0.025     -0.852      0.403      -0.073       0.031
207            0.0241      0.034      0.714      0.483      -0.046       0.094
208           -0.0699      0.042     -1.659      0.111      -0.157       0.018
572            0.0345      0.029      1.182      0.250      -0.026       0.095
801            0.0148      0.033      0.448      0.659      -0.054       0.083
805            0.0813      0.027      3.000      0.007       0.025       0.138
808            0.0226      0.035      0.651      0.522      -0.049       0.095
814            0.0117      0.025      0.469      0.644      -0.040       0.063
842            0.0053      0.026      0.203      0.841      -0.049       0.060
867           -0.0095      0.034     -0.280      0.782      -0.080       0.061
983            0.0057      0.018      0.317      0.754      -0.031       0.043
998           -0.0248      0.042     -0.598      0.556      -0.111       0.061
1026           0.0170      0.011      1.504      0.147      -0.006       0.040
1027          -0.0315      0.023     -1.370      0.185      -0.079       0.016
1147          -0.0044      0.032     -0.136      0.893      -0.071       0.062
1173           0.0110      0.034      0.319      0.752      -0.061       0.083
1175           0.0015      0.050      0.029      0.977      -0.102       0.105
1211          -0.0113      0.024     -0.467      0.645      -0.062       0.039
1213          -0.0472      0.042     -1.116      0.277      -0.135       0.041
1385          -0.0493      0.043     -1.155      0.261      -0.138       0.039
1445           0.0032      0.033      0.096      0.924      -0.065       0.072
1950           0.0118      0.023      0.507      0.617      -0.037       0.060
1956          -0.0207      0.010     -2.088      0.049      -0.041      -0.000
2060          -0.1096      0.052     -2.090      0.048      -0.218      -0.001
2308           0.0061      0.021      0.298      0.769      -0.037       0.049
23

In [20]:
print(bonferroni_correction(model.pvalues, alpha=0.05))

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([  5.06840256,  47.60953626,  14.86290717,  81.05458141,
        54.68706095,  31.41998312,  21.17450173,  10.13547604,
        83.97572787,  16.15018927,  14.39431539,  40.7389554 ,
        48.762267

In [21]:
listEGFR = BRCAE.columns.intersection(lsEGFR)
BRCAEEGFR = BRCAE[listEGFR]
BRCAMEGFR = BRCAM[listEGFR]
BRCACEGFR = BRCAC[listEGFR]  

In [22]:
X = BRCAEEGFR
y = PredBRCA.detach().numpy()

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     6893.
Date:                Sat, 12 Jan 2019   Prob (F-statistic):               0.00
Time:                        18:16:54   Log-Likelihood:                 2466.9
No. Observations:                 921   AIC:                            -4732.
Df Residuals:                     820   BIC:                            -4245.
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
102            0.0011      0.002      0.672      0.502      -0.002       0.004
107           -0.0011      0.000     -2.468      0.014      -0.002      -0.000
108           -0.0013      0.001     -2.000      0.046      -0.003   -2.48e-05
109            0.0002      0.001      0.183      0.855      -0.002       0.003
111           -0.0003      0.001     -0.611      0.541      -0.001       0.001
112            0.0015      0.001      1.019      0.309      -0.001       0.004
113           -0.0020      0.001     -1.351      0.177      -0.005       0.001
114           -0.0026      0.002     -1.382      0.167      -0.006       0.001
115            0.0008      0.001      0.651      0.515      -0.002       0.003
160            0.0067      0.002      3.978      0.000       0.003       0.010
161           -0.0014      0.002     -0.675      0.500      -0.006       0.003
163         5.495e-05      0.001      0.047      0.962      -0.002       0.002
207            0.0028      0.002      1.587      0.113      -0.001       0.006
208           -0.0023      0.002     -1.390      0.165      -0.006       0.001
572           -0.0023      0.002     -1.392      0.164      -0.005       0.001
801            0.0022      0.002      1.102      0.271      -0.002       0.006
805            0.0063      0.002      3.883      0.000       0.003       0.010
808            0.0019      0.002      0.939      0.348      -0.002       0.006
814            0.0010      0.002      0.553      0.580      -0.003       0.004
842           -0.0025      0.002     -1.337      0.182      -0.006       0.001
867           -0.0038      0.002     -1.849      0.065      -0.008       0.000
983         1.694e-05      0.001      0.019      0.985      -0.002       0.002
998            0.0023      0.002      0.999      0.318      -0.002       0.007
1026          -0.0022      0.001     -2.660      0.008      -0.004      -0.001
1027           0.0012      0.001      1.529      0.127      -0.000       0.003
1147           0.0039      0.002      2.011      0.045    9.45e-05       0.008
1173          -0.0016      0.002     -0.793      0.428      -0.006       0.002
1175           0.0050      0.002      2.424      0.016       0.001       0.009
1211           0.0079      0.002      4.775      0.000       0.005       0.011
1213           0.0011      0.001      0.815      0.415      -0.002       0.004
1385          -0.0048      0.003     -1.743      0.082      -0.010       0.001
1445          -0.0036      0.002     -1.679      0.094      -0.008       0.001
1950           0.0010      0.001      1.717      0.086      -0.000       0.002
1956           0.0041      0.001      5.366      0.000       0.003       0.006
2060          -0.0023      0.002     -1.028      0.304      -0.007       0.002
2308          -0.0014      0.002     -0.885      0.377      -0.004       0.002
23

In [23]:
print(bonferroni_correction(model.pvalues, alpha=0.05))

(array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([5.06825579e+01, 1.39415496e+00, 4.62728727e+00, 8.63416692e+01,
       5.46703973e+01, 3.11693984e+01, 1.78878580e+01, 1.69076294e+01,
       5.20628937e+01, 7.63775258e-03, 5.05012625e+01, 9.7174779

In [24]:
listEGFR = PAADE.columns.intersection(lsEGFR)
PAADEEGFR = PAADE[listEGFR]
PAADMEGFR = PAADM[listEGFR]
PAADCEGFR = PAADC[listEGFR]   

In [25]:
X = PAADEEGFR
y = PredPAAD.detach().numpy()

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     251.5
Date:                Sat, 12 Jan 2019   Prob (F-statistic):           1.78e-29
Time:                        18:17:00   Log-Likelihood:                 364.21
No. Observations:                 130   AIC:                            -526.4
Df Residuals:                      29   BIC:                            -236.8
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
102           -0.0263      0.023     -1.162      0.255      -0.072       0.020
107            0.0055      0.013      0.414      0.682      -0.022       0.032
108           -0.0179      0.025     -0.717      0.479      -0.069       0.033
109            0.0112      0.019      0.600      0.553      -0.027       0.049
111            0.0037      0.013      0.282      0.780      -0.023       0.031
112           -0.0120      0.023     -0.527      0.602      -0.058       0.034
113            0.0143      0.014      1.031      0.311      -0.014       0.043
114           -0.0118      0.039     -0.302      0.765      -0.092       0.068
115            0.0008      0.036      0.022      0.983      -0.072       0.074
160           -0.0357      0.032     -1.114      0.274      -0.101       0.030
161           -0.0005      0.027     -0.017      0.987      -0.056       0.056
163            0.0254      0.031      0.828      0.415      -0.037       0.088
207            0.0099      0.031      0.323      0.749      -0.053       0.073
208           -0.0149      0.016     -0.959      0.346      -0.047       0.017
572            0.0064      0.024      0.267      0.791      -0.042       0.055
801            0.0163      0.040      0.408      0.686      -0.065       0.098
805            0.0394      0.032      1.232      0.228      -0.026       0.105
808            0.0136      0.037      0.367      0.716      -0.062       0.089
814           -0.0174      0.015     -1.129      0.268      -0.049       0.014
842           -0.0067      0.021     -0.327      0.746      -0.049       0.035
867           -0.0127      0.029     -0.442      0.661      -0.072       0.046
983            0.0013      0.015      0.089      0.930      -0.029       0.032
998           -0.0207      0.043     -0.482      0.633      -0.109       0.067
1026          -0.0120      0.014     -0.878      0.387      -0.040       0.016
1027           0.0094      0.025      0.382      0.705      -0.041       0.060
1147          -0.0039      0.035     -0.111      0.912      -0.075       0.068
1173           0.0050      0.041      0.121      0.905      -0.079       0.089
1175           0.0105      0.033      0.318      0.753      -0.057       0.078
1211          -0.0214      0.015     -1.443      0.160      -0.052       0.009
1213           0.0339      0.041      0.830      0.413      -0.050       0.117
1385          -0.0315      0.034     -0.915      0.367      -0.102       0.039
1445           0.0012      0.022      0.056      0.956      -0.044       0.046
1950         2.53e-05      0.011      0.002      0.998      -0.023       0.023
1956           0.0078      0.017      0.457      0.651      -0.027       0.043
2060           0.0024      0.044      0.054      0.958      -0.087       0.092
2308          -0.0160      0.020     -0.788      0.437      -0.057       0.025
23

In [26]:
print(bonferroni_correction(model.pvalues, alpha=0.05))

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([ 25.71654309,  68.88518558,  48.36811448,  55.88769998,
        78.77410576,  60.81762683,  31.41328896,  77.27234893,
        99.26457938,  27.69865112,  99.63968843,  41.88315265,
        75.667971

In [27]:
listEGFR = LUADE.columns.intersection(lsEGFR)
LUADEEGFR = LUADE[listEGFR]
LUADMEGFR = LUADM[listEGFR]
LUADCEGFR = LUADC[listEGFR]

In [28]:
X = LUADEEGFR
y = PredLUAD.detach().numpy()

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     1895.
Date:                Sat, 12 Jan 2019   Prob (F-statistic):               0.00
Time:                        18:17:07   Log-Likelihood:                 1160.5
No. Observations:                 475   AIC:                            -2119.
Df Residuals:                     374   BIC:                            -1699.
Df Model:                         101                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
102            0.0028      0.003      0.825      0.410      -0.004       0.010
107           -0.0021      0.002     -1.161      0.247      -0.006       0.001
108            0.0013      0.003      0.412      0.681      -0.005       0.007
109            0.0013      0.003      0.524      0.600      -0.004       0.006
111           -0.0006      0.002     -0.411      0.681      -0.004       0.002
112            0.0020      0.003      0.771      0.441      -0.003       0.007
113           -0.0034      0.002     -1.652      0.099      -0.007       0.001
114           -0.0046      0.004     -1.172      0.242      -0.012       0.003
115            0.0020      0.003      0.798      0.426      -0.003       0.007
160            0.0059      0.005      1.221      0.223      -0.004       0.015
161           -0.0005      0.004     -0.128      0.898      -0.007       0.007
163            0.0038      0.003      1.226      0.221      -0.002       0.010
207           -0.0066      0.004     -1.705      0.089      -0.014       0.001
208            0.0001      0.004      0.033      0.974      -0.008       0.008
572           -0.0035      0.003     -1.009      0.314      -0.010       0.003
801           -0.0123      0.004     -3.097      0.002      -0.020      -0.004
805            0.0078      0.004      2.175      0.030       0.001       0.015
808            0.0009      0.005      0.193      0.847      -0.008       0.010
814           -0.0018      0.004     -0.500      0.617      -0.009       0.005
842            0.0039      0.004      1.104      0.270      -0.003       0.011
867            0.0006      0.004      0.139      0.890      -0.008       0.009
983            0.0038      0.002      1.848      0.065      -0.000       0.008
998            0.0167      0.005      3.572      0.000       0.008       0.026
1026          -0.0011      0.002     -0.589      0.556      -0.005       0.003
1027           0.0030      0.003      1.039      0.300      -0.003       0.009
1147           0.0067      0.005      1.436      0.152      -0.002       0.016
1173           0.0047      0.004      1.070      0.285      -0.004       0.013
1175          -0.0014      0.005     -0.270      0.787      -0.012       0.009
1211           0.0088      0.003      2.973      0.003       0.003       0.015
1213          -0.0055      0.004     -1.259      0.209      -0.014       0.003
1385          -0.0124      0.005     -2.316      0.021      -0.023      -0.002
1445          -0.0057      0.004     -1.319      0.188      -0.014       0.003
1950           0.0016      0.001      1.287      0.199      -0.001       0.004
1956           0.0073      0.001      5.165      0.000       0.005       0.010
2060          -0.0115      0.005     -2.353      0.019      -0.021      -0.002
2308           0.0014      0.003      0.494      0.621      -0.004       0.007
23

In [29]:
print(bonferroni_correction(model.pvalues, alpha=0.05))

(array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False]), array([4.14015235e+01, 2.48985067e+01, 6.87655871e+01, 6.06407957e+01,
       6.88273009e+01, 4.45728983e+01, 1.00290905e+01, 2.44236844e+01,
       4.29890891e+01, 2.25237889e+01, 9.06980328e+01, 2.2333854